In [3]:
import wpspecdev
from matplotlib import pyplot as plt
import numpy as np

# define a test structure
test_args = {
    "therml": True,
    'TeMperature' : 440,
    "wavelength_list": [300e-9, 20300e-9, 5001],
    "material_list": ['Air', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'polystyrene', 'SiO2', 'Al', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'Ta2O5', 'SiO2', 'Air'], 
    "thickness_list": [0, 2.6422605026007024e-07, 1.5e-09, 2.6422605026007024e-07, 3.733648095151694e-07, 5.284521005201405e-07, 3.733648095151694e-07, 5.284521005201405e-07, 3.733648095151694e-07, 0.0001, 0.001, 3.8014616788692957e-07, 1.5e-09, 3.8014616788692957e-07, 5.311796118517261e-07, 7.602923357738591e-07, 5.311796118517261e-07, 7.602923357738591e-07, 5.311796118517261e-07, 0.0001, 0]
}




We are going to compute the transmissivity of the above structure to the AM1.5 solar spectrum that falls below the bandgap wavelength (assumed to be $\lambda_{bg} = 760$ nm (see [here](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.92.045414)
and [here](https://pubs.acs.org/doi/10.1021/acs.nanolett.6b03114))
and the thermal emission in the SWIR ($\lambda_{min} = 1000$ nm  and $\lambda_{max} =  3500$ nm) assuming an emitter temperature of $T_E = 440 K$.

In particular, we will compute

$$ T_{AM1.5} = \int_0^{\lambda_{bg}} I_{AM1.5}(\lambda) T(\lambda) d\lambda $$

and

$$ \epsilon_{SWIR} = \int_{\lambda_{min}}^{\lambda_{max}} B(\lambda, T_E) \epsilon(\lambda) d\lambda $$

In [6]:
import time
start = time.time()

sf = wpspecdev.SpectrumFactory()  
# create instance of class
ts = sf.spectrum_factory("Tmm", test_args)

# want to create a step function so that we can only integrate
# a subinterval of the entire reflectivity function up to 760 nm
target_pv = 760e-9 * np.ones_like(ts.wavelength_array)
stop_idx = np.argmin(np.abs( target_pv - ts.wavelength_array) )

# step function to cut off at \lambda_bg of the PV
step_pv = np.zeros_like(ts.wavelength_array)
step_pv[:stop_idx] = 1.

# do the same for the to create a step function between 1000 and 3500 nm
target_min = 1000e-9 * np.ones_like(ts.wavelength_array)
target_max = 3500e-9 * np.ones_like(ts.wavelength_array)

min_idx = np.argmin(np.abs( target_min - ts.wavelength_array))
max_idx = np.argmin(np.abs( target_max - ts.wavelength_array))

step_swir = np.zeros_like(ts.wavelength_array)
step_swir[min_idx:max_idx] = 1.


### Integrate the above-gap solar flux transmitted through optic
AM_transmit_power = np.trapz(ts._solar_spectrum * step_pv *  ts.transmissivity_array, ts.wavelength_array)
SWIR_emit_power = np.trapz(ts.thermal_emission_array * step_swir, ts.wavelength_array)

end = time.time()
print("time required")
print(end-start)
print(SWIR_emit_power,  " W / s / m^2 emitted")
print(AM_transmit_power," W / s / m^2 transmitted")



getting solar spectrum
 Your spectra have been computed! 😎 
 Your spectra have been computed! 🔥 
175
800
time required
1.4325051307678223
6.585462586251555  W / s / m^2 emitted
284.7632468333843  W / s / m^2 transmitted
